In [2]:
import os

os.chdir('../')

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


In [4]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [5]:
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [6]:
# too slow -> from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github
import optuna

def objective_function(trial):
    
    '''recommender_implicitAls.fit(iterations= int(83.64347328971459), factors= int(125.3856187337561), regularization= 0.05106206327352681, use_gpu=False, num_threads=0, confidence_scaling=linear_scaling_confidence,**{"alpha":5.579621138255679})
'''
    
    factors = trial.suggest_int("factors", 1, 400)
    iterations = trial.suggest_int("iterations", 1, 500)
    regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
    alpha = trial.suggest_uniform("alpha", 0.0, 50.0)
    
    print(f"Current parameters: factors={factors}, iterations={iterations}, regularization={regularization}, alpha={alpha}")
    recommender = ImplicitALSRecommender(URM_train_validation)
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/IALS/IALS_first_trial.csv", index = False)

optuna_study = optuna.create_study(study_name="IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-27 17:55:21,627] A new study created in memory with name: IALS


Current parameters: factors=122, iterations=203, regularization=0.1149551173878709, alpha=31.40168583772485


/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1494389866.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1494389866.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 50.0)
/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having

  0%|          | 0/203 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 35595 (100.0%) in 19.90 sec. Users per second: 1788


...

Current parameters: factors=312, iterations=208, regularization=0.41682701467153505, alpha=11.55775932033436


/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1494389866.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.0, 1.0)
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1494389866.py:13: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 50.0)


  0%|          | 0/208 [00:00<?, ?it/s]

[W 2024-12-28 15:54:44,718] Trial 23 failed with parameters: {'factors': 312, 'iterations': 208, 'regularization': 0.41682701467153505, 'alpha': 11.55775932033436} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1494389866.py", line 17, in objective_function
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/MatrixFactorization/ImplicitALSRecommender.py", line 76, in fit
    use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py", l

KeyboardInterrupt: 

# Too slow
First let's verify it isn't just a problem of more epochs. Then in case let's speed up the convergence

In [7]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.03834431445472179, 'factors': 397.0, 'iterations': 105.0, 'regularization': 0.3707848450257657, 'alpha': 4.815638211952356}


In [12]:
params = optuna_study.best_params
params["iterations"] = 300
print("New Parameters: ", params)

New Parameters:  {'factors': 397, 'iterations': 300, 'regularization': 0.3707848450257657, 'alpha': 4.815638211952356}


In [13]:
recommender = ImplicitALSRecommender(URM_train_validation)
recommender.fit(**params)

  0%|          | 0/300 [00:00<?, ?it/s]

In [14]:
result_dict, _ = evaluator_test.evaluateRecommender(recommender)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

EvaluatorHoldout: Processed 35595 (100.0%) in 22.50 sec. Users per second: 1582
0.038388545519812474


# Nevermind, using colab with gpu :(

Doing some tests after some experiments with colab

In [33]:
trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=150, factors=985, regularization=0.05358304457765484, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":11.70840956665871})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/150 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 25.85 sec. Users per second: 1377
0.04497427296057561


In [45]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=985, regularization=0.7, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":7})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 27.63 sec. Users per second: 1288
0.045899234554521294


In [43]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=985, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 27.34 sec. Users per second: 1302
0.04593349899776172


In [49]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1000, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 25.70 sec. Users per second: 1385
0.04609735962559245


In [52]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1250, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 27.99 sec. Users per second: 1272
0.04689272057560994


In [54]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1350, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 29.52 sec. Users per second: 1206
0.04757753898019192


In [55]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1450, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 28.86 sec. Users per second: 1233
0.047689126125702884


In [56]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1550, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 28.78 sec. Users per second: 1237
0.048148553947962094


In [57]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1650, regularization=0.3, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 29.88 sec. Users per second: 1191
0.048254146627511894


In [61]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1650, regularization=0.4, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 31.17 sec. Users per second: 1142
0.048276074310417236


In [62]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_train_validation)
trial_recomm.fit(iterations=10, factors=1650, regularization=0.5, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
result_dict, _ = evaluator_test.evaluateRecommender(trial_recomm)
MAP = result_dict.loc[10]["MAP"]
print(MAP)

  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35595 (100.0%) in 30.34 sec. Users per second: 1173
0.04828206320220468


Not finding anything better...

In [63]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github

trial_recomm = ImplicitALSRecommender(URM_all)
trial_recomm.fit(iterations=10, factors=1650, regularization=0.5, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":10})

target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(trial_recomm.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/IALS_implicit_iterations=10, factors=1650, regularization=0.5_alpha_10_MAP:0,0482.csv", index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9


In [64]:
import gc
gc.collect()

0

# Back on Mac
Since we don't need huge amounts of epochs, we can go back running optuna here. This time I want to try optimizing only alpha and factors

In [65]:
# too slow -> from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github
import optuna

def objective_function(trial):
    
    '''recommender_implicitAls.fit(iterations= int(83.64347328971459), factors= int(125.3856187337561), regularization= 0.05106206327352681, use_gpu=False, num_threads=0, confidence_scaling=linear_scaling_confidence,**{"alpha":5.579621138255679})
'''
    
    factors = trial.suggest_int("factors", 1350, 1950)  # Refined range for factors
    #regularization = trial.suggest_uniform("regularization", 0.0, 1.0)  # Refined range for regularization
    alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
    iterations = 10
    regularization = trial.suggest_uniform("regularization", 0.1, 0.8)

    
    print(f"Current parameters: factors={factors}, iterations={iterations}, regularization={regularization}, alpha={alpha}")
    recommender = ImplicitALSRecommender(URM_train_validation)
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/IALS/IALS_restart_first.csv", index = False)

optuna_study = optuna.create_study(study_name="IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-29 21:26:23,780] A new study created in memory with name: IALS
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1249916818.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1249916818.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.1, 0.8)


Current parameters: factors=1610, iterations=10, regularization=0.3106873884636875, alpha=1.2237073580995639


  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 35595 (100.0%) in 32.60 sec. Users per second: 1092


[I 2024-12-29 21:33:57,285] Trial 0 finished with value: 0.0454691837403578 and parameters: {'factors': 1610, 'alpha': 1.2237073580995639, 'regularization': 0.3106873884636875}. Best is trial 0 with value: 0.0454691837403578.


...

Current parameters: factors=1912, iterations=10, regularization=0.4789464099283705, alpha=6.884635910690805


/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1249916818.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1249916818.py:14: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.1, 0.8)


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-12-30 01:03:29,308] Trial 25 failed with parameters: {'factors': 1912, 'alpha': 6.884635910690805, 'regularization': 0.4789464099283705} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1249916818.py", line 19, in objective_function
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/MatrixFactorization/ImplicitALSRecommender.py", line 76, in fit
    use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py", line 170, in fit
   

KeyboardInterrupt: 

# We could use even more factors

In [66]:
import gc
gc.collect()

1072

In [67]:
# too slow -> from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github
import optuna

def objective_function(trial):
    
    factors = trial.suggest_int("factors", 1550, 2550)  # Refined range for factors
    #regularization = trial.suggest_uniform("regularization", 0.0, 1.0)  # Refined range for regularization
    alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
    iterations = 10
    regularization = trial.suggest_uniform("regularization", 0.1, 0.8)

    
    print(f"Current parameters: factors={factors}, iterations={iterations}, regularization={regularization}, alpha={alpha}")
    recommender = ImplicitALSRecommender(URM_train_validation)
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/IALS/IALS_restart_second.csv", index = False)

optuna_study = optuna.create_study(study_name="IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 01:04:04,209] A new study created in memory with name: IALS
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1016136916.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1016136916.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.1, 0.8)


Current parameters: factors=1620, iterations=10, regularization=0.39867791536626407, alpha=12.848834943026686


  0%|          | 0/10 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 35595 (100.0%) in 35.12 sec. Users per second: 1014


[I 2024-12-30 01:14:14,427] Trial 0 finished with value: 0.04793973426800887 and parameters: {'factors': 1620, 'alpha': 12.848834943026686, 'regularization': 0.39867791536626407}. Best is trial 0 with value: 0.04793973426800887.


...

Current parameters: factors=2514, iterations=10, regularization=0.5687047485194247, alpha=4.471317069433278


/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1016136916.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1016136916.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.1, 0.8)


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-12-30 09:10:51,028] Trial 32 failed with parameters: {'factors': 2514, 'alpha': 4.471317069433278, 'regularization': 0.5687047485194247} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/1016136916.py", line 16, in objective_function
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/MatrixFactorization/ImplicitALSRecommender.py", line 76, in fit
    use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py", line 163, in fit
   

KeyboardInterrupt: 

# Factors can be increased even more

In [69]:
# too slow -> from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender # Credit Nicola Cesare Github
import optuna

def objective_function(trial):
    
    factors = trial.suggest_int("factors", 2300, 4000)  # Refined range for factors
    #regularization = trial.suggest_uniform("regularization", 0.0, 1.0)  # Refined range for regularization
    alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
    iterations = 10
    regularization = trial.suggest_uniform("regularization", 0.1, 0.8)

    
    print(f"Current parameters: factors={factors}, iterations={iterations}, regularization={regularization}, alpha={alpha}")
    recommender = ImplicitALSRecommender(URM_train_validation)
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})

    result_dict, _ = evaluator_test.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP


class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/IALS/IALS_restart_third.csv", index = False)

optuna_study = optuna.create_study(study_name="IALS", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100,
                      )

[I 2024-12-30 09:11:17,577] A new study created in memory with name: IALS
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/4107685421.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.0, 20.0)  # Refined range for alpha
/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/4107685421.py:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  regularization = trial.suggest_uniform("regularization", 0.1, 0.8)


Current parameters: factors=3484, iterations=10, regularization=0.41929558393279837, alpha=19.58419802629827


  0%|          | 0/10 [00:00<?, ?it/s]

[W 2024-12-30 09:27:02,980] Trial 0 failed with parameters: {'factors': 3484, 'alpha': 19.58419802629827, 'regularization': 0.41929558393279837} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/50/7hjr20g17_18p725nbt0qqv00000gn/T/ipykernel_81509/4107685421.py", line 16, in objective_function
    recommender.fit(iterations=iterations, factors=factors, regularization=regularization, use_gpu=False, num_threads=0, confidence_scaling="linear", **{"alpha":alpha})
  File "/Users/lucapagano/Developer/RecSys/Competizione/Recommenders/MatrixFactorization/ImplicitALSRecommender.py", line 76, in fit
    use_native=use_native, use_cg=use_cg, use_gpu=use_gpu,
  File "/Users/lucapagano/miniconda3/envs/RecSysPD/lib/python3.8/site-packages/implicit/cpu/als.py", line 170, in fit
   

KeyboardInterrupt: 